# Capstone Project - The Battle of the Neighborhoods (Week 2)
###  Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>
The project named The battle of neighbourhood deals with finding a location for the restaurant,ie a suitable location.We have to build a restaurant in Stockholm,Sweden. Inorder to build a restaurant we have to analyse the location and restuarant and we have to locate the areas which have less resturants,ie least crowded area.Inorder to get profit in an area,We have to locate areas with least or no restuarants nearby.
We will collect maps and data and will use our data science to find the appropiate location.


## Data <a name="data"></a>
Data title: - Open Data published by Government of Sweden under the section HM Land Registry: Location Data Type of data: Dataset in form of CSV file Duration: August 2018 data

Location Data includes information on all property sales in Sweden and Stockholm that are sold for full market value and are lodged with us for registration. The dataset includes the transactions received at HM Land Registry in the period from the first to the last day of August 2018.

• Data title: Google Maps Geocoding API

Type of data: JSON

Duration: N/A

Description of the data: Location coordinates obtained by Gmaps API calls. Location Information obtained is used to obtain the location coordinates from Google Maps. A separate Python script has been developed to extract the unique street names, district names from the Dataset and embed those in the GMaps API calls to obtain the required information.

Google Cloud Platform/ Google Maps• Data title: Restaurant location data

Description of the data: Location coordinates obtained by Foursquare API calls. To determine the proximity of various amenities as per the client’s requirement, restaurant location data is used.

### IMPORTING OF NECESSERY PACKAGES

In [2]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml
!conda install -c conda-forge folium=0.5.0 --yes 

import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [5]:
!pip install pyproj
import pyproj

     |████████████████████████████████| 11.2MB 1.9MB/s eta 0:00:01


In [8]:
import math

### LATITUDE AND LONGITUDE IMPORTING

In [3]:
address = 'Stockholm, Sweden'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
stockholm_center = [latitude, longitude]
print('The geograpical coordinate of ', address ,'are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Stockholm, Sweden are 59.3251172, 18.0710935.


In [6]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Stockholm center longitude={}, latitude={}'.format(stockholm_center[1], stockholm_center[0]))
x, y = lonlat_to_xy(stockholm_center[1], stockholm_center[0])
print('Stockholm center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Stockholm center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Stockholm center longitude=18.0710935, latitude=59.3251172
Stockholm center UTM X=674735.1349179444, Y=6580284.089114029
Stockholm center longitude=18.0710935, latitude=59.3251172


In [9]:
stockholm_center_x, stockholm_center_y = lonlat_to_xy(stockholm_center[1], stockholm_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = stockholm_center_x - 6000
x_step = 600
y_min = stockholm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(stockholm_center_x, stockholm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


In [10]:
map_stockholm = folium.Map(location=stockholm_center, zoom_start=13)
folium.Marker(stockholm_center, popup='Stockholm').add_to(map_stockholm)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_stockholm)
    #folium.Marker([lat, lon]).add_to(map_Stockholm)
map_stockholm

In [11]:
CLIENT_ID = 'JA2RLBI4IPPSFTWAVNIXIOLLUVQCOHIB4AV0ZTEBSLZGE0GI' # your Foursquare ID
CLIENT_SECRET = 'IUE4XAZ1BIFVBZDALGNDY4HDW3IU1EDX4NAGN4F3OW4VWAE5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JA2RLBI4IPPSFTWAVNIXIOLLUVQCOHIB4AV0ZTEBSLZGE0GI
CLIENT_SECRET:IUE4XAZ1BIFVBZDALGNDY4HDW3IU1EDX4NAGN4F3OW4VWAE5


In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JA2RLBI4IPPSFTWAVNIXIOLLUVQCOHIB4AV0ZTEBSLZGE0GI&client_secret=IUE4XAZ1BIFVBZDALGNDY4HDW3IU1EDX4NAGN4F3OW4VWAE5&v=20180605&ll=59.3251172,18.0710935&radius=500&limit=100'

In [57]:
results = requests.get(url).json()

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
#pull the actual data from the Foursquare API

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,Science Fiction Bokhandeln,4b06fe89f964a520fef422e3,Bookstore,59.324047,18.070682
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
2,Stortorget,4b0e3607f964a520fa5523e3,Plaza,59.324973,18.070727
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
4,Stockholms Gästabud,4e8596c602d59e1a9afa5d5e,Gastropub,59.325569,18.073991
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958


In [56]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [17]:
nearby_venues['categories'].unique()

array(['Bookstore', 'Cocktail Bar', 'Plaza', 'Burger Joint', 'Gastropub',
       'Scandinavian Restaurant', 'Wine Bar', 'Deli / Bodega', 'Pub',
       'Jazz Club', 'Bakery', 'Comic Shop', 'Palace', 'Record Shop',
       'Polish Restaurant', 'Mexican Restaurant', 'Café', 'Historic Site',
       'Theme Restaurant', 'Noodle House', 'Bike Shop',
       'Outdoor Sculpture', 'Tea Room', 'History Museum', 'Irish Pub',
       'Hostel', 'Ice Cream Shop', 'Italian Restaurant', 'Coffee Shop',
       'Gift Shop', 'French Restaurant', 'German Restaurant',
       'Toy / Game Store', 'Bar', 'Restaurant', 'Fish & Chips Shop',
       'Seafood Restaurant', 'Hotel', 'Museum', 'Sushi Restaurant',
       'Church', 'Candy Store', 'Art Museum', 'Opera House',
       'Monument / Landmark', 'Park', 'Breakfast Spot', 'Boat or Ferry',
       'Other Great Outdoors', 'Building'], dtype=object)

In [18]:
removal_list = ['Bookstore', 'Bakery', 'Jazz Club', "Bike Shop", 'Comic Shop', 'Plaza', 'Gastropub',
               'Record Shop', 'Pub', 'Pet Store', 'Historic Site', 'Outdoor Sculpture', 'Tea Room', 'History Museum', 'Irish Pub',
       'Hostel','Gift Shop','BBQ Joint','Toy / Game Store', 'Bar','Fish & Chips Shop','Hotel', 'Museum','Church', 'Candy Store', 'Art Museum', 'Opera House',
       'Monument / Landmark', 'Park', 'Breakfast Spot', 'Boat or Ferry',
       'Other Great Outdoors', 'Building']

nearby_venues2 = nearby_venues.copy()


#getting a clear dataframe of just restaurants
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2

,name,id,categories,lat,lng
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140


In [19]:
venue_id_list = nearby_venues2['id'].tolist()
venue_id_list

['51434b65e4b00cf8d47795f1',
 '549447fb498ed7d33f190dc2',
 '534d0a34498ef782062ac89c',
 '514c0868e4b07a3d6af2fca3',
 '522784bb11d2164a04286219',
 '4ba0c261f964a520a77b37e3',
 '537f7e60498e54e075c24907',
 '4b9a5e8ff964a52072af35e3',
 '51ccb742498e64f625f8ce57',
 '4adcdaeef964a520c95a21e3',
 '5729f110498e722233305953',
 '4c78322f93ef236a9772ab0f',
 '550867d7498e86f0d8c9a61d',
 '50c0cc79e4b0a80fdc3460df',
 '4ece875cb634dd298f5ea304',
 '4dc28371887770a6be00accd',
 '4b63079df964a520945e2ae3',
 '58c90efcd243323eaf526efb',
 '583875a96d632566864f820c',
 '4d381d443ffba143aed55856',
 '4c63fd4479d1e21eacddd915',
 '4b180045f964a52039cb23e3',
 '4adcdaedf964a520555a21e3',
 '4e198d6e18a8166f7396ffaa',
 '4adcdaf0f964a520655b21e3',
 '4adcdaebf964a520e35921e3',
 '5731e745498ea03b4bfe08cb',
 '51cb1bd3498e9a51850326c1',
 '4d66061b48ee6ea85d166168',
 '4b583319f964a520bf4d28e3',
 '4c9cb5199975a143948034cf',
 '4b9a284ef964a520d7a135e3',
 '55df60fd498e44d120c585cb',
 '51829a91498e80962fe0411e',
 '4b4f5698f964

In [21]:
#set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)

[198, 448, 96, 101, 40, 128, 101, 109, 178, 498, 27, 120, 75, 69, 271, 33, 144, 14, 24, 104, 76, 112, 99, 217, 114, 221, 23, 30, 14, 35, 34, 45, 8, 152, 36, 53, 14, 19, 64, 57, 60, 20, 24, 31, 8]


In [22]:

#double check that we did not lose any venues based on if likes were available

print(len(like_list))
print(len(venue_id_list))

45
45


In [23]:
stockholm_venues = nearby_venues2.copy()
stockholm_venues.head()

,name,id,categories,lat,lng
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161


In [24]:
# add in the list of likes

stockholm_venues['total likes'] = like_list
stockholm_venues.head()

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40


In [25]:
# now let's bin total likes

print(stockholm_venues['total likes'].max())
print(stockholm_venues['total likes'].min())
print(stockholm_venues['total likes'].median())
print(stockholm_venues['total likes'].mean())

498
8
64.0
96.53333333333333


In [26]:
import matplotlib.pyplot as plt
stockholm_venues['total likes'].hist(bins=4)
plt.show()

<Figure size 640x480 with 1 Axes>

In [28]:
# what are the bins we want to use?
print(np.percentile(stockholm_venues['total likes'], 25))
print(np.percentile(stockholm_venues['total likes'], 50))
print(np.percentile(stockholm_venues['total likes'], 75))

30.0
64.0
114.0


In [29]:
poor = stockholm_venues['total likes']<=24
below_avg = stockholm_venues[(stockholm_venues['total likes']>24) & (stockholm_venues['total likes']<=45)]
abv_avg = stockholm_venues[(stockholm_venues['total likes']>45) & (stockholm_venues['total likes']<=76)]
great = stockholm_venues['total likes']>76
stockholm_venues

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967,128
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958,101
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670,109
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891,178
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140,498


In [30]:
stockholm_venues['categories'].unique()

array(['Cocktail Bar', 'Burger Joint', 'Scandinavian Restaurant',
       'Wine Bar', 'Deli / Bodega', 'Palace', 'Polish Restaurant',
       'Mexican Restaurant', 'Café', 'Theme Restaurant', 'Noodle House',
       'Ice Cream Shop', 'Italian Restaurant', 'Coffee Shop',
       'French Restaurant', 'German Restaurant', 'Restaurant',
       'Seafood Restaurant', 'Sushi Restaurant'], dtype=object)

In [34]:
bars = ['Pub', 'Cocktail Bar', 'Bar', 'Dive Bar', 'Sports Bar']
other = ['Bagel Shop', 'Tea Room', 'Donut Shop', 'Coffee Shop', 'Bubble Tea Shop', 'Sandwich Place', 'Boutique', 'Ice Cream Shop']
euro_asia_indian_food = ['Falafel Restaurant', 'Korean Restaurant','Sushi Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Poke Place', 'Thai Restaurant', 'Vietnamese Restaurant']
mex_southam_food = ['Cuban Restaurant', 'Mexican Restaurant', 'South American Restaurant', 'Latin American Restaurant']
american_food = ['Burger Joint', 'Restaurant', 'American Restaurant']
italian_food = ['Italian Restaurant', 'Seafood Restaurant', 'Pizza Place']

def conditions2(s):
    if s['categories'] in bars:
        return 'bars'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in euro_asia_indian_food:
        return 'euro asia indian food'
    if s['categories'] in mex_southam_food:
        return 'mex southam food'
    if s['categories'] in american_food:
        return 'american food'
    if s['categories'] in italian_food:
        return 'italian food'

In [35]:
stockholm_venues

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967,128
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958,101
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670,109
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891,178
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140,498


In [41]:
stockholm_onehot = pd.get_dummies(stockholm_venues[['categories', 'total likes']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
stockholm_onehot['Name'] = stockholm_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [stockholm_onehot.columns[-1]] + list(stockholm_onehot.columns[:-1])
stockholm_onehot = stockholm_onehot[fixed_columns]
cluster_df = stockholm_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 3, 3, 1, 3, 3, 3, 2, 0], dtype=int32)

In [42]:
stockholm_venues['label'] = kmeans.labels_
stockholm_venues.head()

,name,id,categories,lat,lng,total likes,label
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198,2
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448,0
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96,3
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101,3
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40,1


In [55]:
# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(stockholm_venues['lat'], stockholm_venues['lng'], stockholm_venues['name'], stockholm_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1]

In [50]:
stockholm_venues.loc[stockholm_venues['label']==0]

,name,id,categories,lat,lng,total likes,label
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448,0
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140,498,0


In [49]:
stockholm_venues.loc[stockholm_venues['label']==1]

,name,id,categories,lat,lng,total likes,label
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40,1
18,Matgatan 22,5729f110498e722233305953,Polish Restaurant,59.324454,18.068671,27,1
21,Gaston - Wine Exploring,50c0cc79e4b0a80fdc3460df,Wine Bar,59.323206,18.068640,69,1
23,Caffellini,4dc28371887770a6be00accd,Café,59.323560,18.071373,33,1
27,Kagges,58c90efcd243323eaf526efb,Scandinavian Restaurant,59.323371,18.069441,14,1
28,The Hairy Pig Restaurant,583875a96d632566864f820c,Theme Restaurant,59.323738,18.068742,24,1
43,StikkiNikki,5731e745498ea03b4bfe08cb,Ice Cream Shop,59.324699,18.068485,23,1
45,Corvina Enoteca,51cb1bd3498e9a51850326c1,Italian Restaurant,59.323060,18.071104,30,1
46,CitySallad,4d66061b48ee6ea85d166168,Coffee Shop,59.326088,18.075268,14,1
48,Pastis,4b583319f964a520bf4d28e3,French Restaurant,59.325093,18.073573,35,1


In [51]:
stockholm_venues.loc[stockholm_venues['label']==2]

,name,id,categories,lat,lng,total likes,label
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198,2
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891,178,2
22,AIFUR | krog & bar,4ece875cb634dd298f5ea304,Scandinavian Restaurant,59.323097,18.071993,271,2
39,Under Kastanjen,4e198d6e18a8166f7396ffaa,Café,59.324794,18.072893,217,2
42,Chokladkoppen,4adcdaebf964a520e35921e3,Café,59.325020,18.070727,221,2


In [52]:
stockholm_venues.loc[stockholm_venues['label']==3]

,name,id,categories,lat,lng,total likes,label
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96,3
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101,3
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967,128,3
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958,101,3
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670,109,3
19,Kryp In,4c78322f93ef236a9772ab0f,Scandinavian Restaurant,59.324694,18.070059,120,3
20,Geronimo's FGT,550867d7498e86f0d8c9a61d,Mexican Restaurant,59.325198,18.067725,75,3
24,Ardbeg Embassy,4b63079df964a520945e2ae3,Scandinavian Restaurant,59.323170,18.071904,144,3
29,Café Schweizer Konditori,4d381d443ffba143aed55856,Café,59.326174,18.068824,104,3
30,Café Stiernan,4c63fd4479d1e21eacddd915,Noodle House,59.323406,18.073597,76,3
